In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import random
import nltk
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup


from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix, classification_report
import matplotlib.pyplot as plt





In [2]:
###Calling the subreddit data

filename = "c:/users/chung/desktop/springboard_stuff/r-depression/r-dep data cleaned.csv"
A = pd.read_csv(filename)
filename_2 = "c:/users/chung/desktop/springboard_stuff/r-ptsd/r-ptsd data cleaned.csv"
B = pd.read_csv(filename_2)
filename_3 = "c:/users/chung/desktop/springboard_stuff/r-cptsd/r-cptsd data cleaned.csv"
C = pd.read_csv(filename_3)
filename_4 = "c:/users/chung/desktop/springboard_stuff/r-bpd/r-bpd data cleaned.csv"
D = pd.read_csv(filename_4)
filename_5 = "c:/users/chung/desktop/springboard_stuff/r-bipolar/r-bipolar data cleaned.csv"
E = pd.read_csv(filename_5)
filename_6 = "c:/users/chung/desktop/springboard_stuff/r-dissociation/r-diss data cleaned.csv"
F = pd.read_csv(filename_6)

###Combining the dataset with the appropriate labels
# 0 = depression
# 1 = ptsd
# 2 = cptsd
# 3 = bpd
# 4 = bipolar
# 5 = dissociation

A_ = A.dropna()
B_ = B.dropna()
C_ = C.dropna()
D_ = D.dropna()
E_ = E.dropna()
F_ = F.dropna()

master = A_.append([B_,C_,D_,E_,F_])
master_df = pd.DataFrame(master)
master_df.to_csv('master.csv', index=False)

'''assigning labels and storing in variable to prepare for holdout creation'''
df_dep = master_df.loc[master_df['label'] == 0] 
df_ptsd = master_df.loc[master_df['label'] == 1]
df_cptsd = master_df.loc[master_df['label'] == 2] 
df_bpd = master_df.loc[master_df['label'] == 3]
df_bipolar = master_df.loc[master_df['label'] == 4] 
df_diss = master_df.loc[master_df['label'] == 5]


'''creating holdouts'''
df_dep_holdout = df_dep.iloc[:100]
df_ptsd_holdout = df_ptsd.iloc[:100]
df_cptsd_holdout = df_cptsd.iloc[:100]
df_bpd_holdout = df_bpd.iloc[:100]
df_bipolar_holdout = df_bipolar.iloc[:100]
df_diss_holdout = df_diss.iloc[:100]


'''re-defining variables with holdout documents ommitted'''
df_dep = df_dep.iloc[100:]
df_ptsd = df_ptsd.iloc[100:]
df_cptsd = df_cptsd.iloc[100:]
df_bpd = df_bpd.iloc[100:]
df_bipolar = df_bipolar.iloc[100:]
df_diss = df_diss.iloc[100:]


'''concat the pieces back together'''
dfnew = pd.concat([df_dep, df_ptsd, df_cptsd, df_bpd, df_bipolar, df_diss])
df_holdout = pd.concat([df_dep_holdout, df_ptsd_holdout, df_cptsd_holdout, df_bpd_holdout, df_bipolar_holdout, df_diss_holdout])


dfnew.head(10)


,text,label
100,"['struggling', 'sad', 'really', 'dark', 'thoug...",0
101,"['something', 'broke', 'inside', 'never', 'gon...",0
102,"['get', 'anything', 'done', 'motivation', 'wis...",0
103,"['go', 'college', 'make', 'parent', 'happy', '...",0
104,"['maybe', 'genetics', 'maybe', 'childhood', 't...",0
105,"['drank', 'yesterday', 'headache', 'cut', 'thi...",0
106,"['little', 'reading', 'functioning', 'depressi...",0
107,"['lost', 'even', 'get', 'house', 'hard', 'get'...",0
108,"['basically', 'depressed', 'year', 'told', 'pa...",0
109,"['everyday', 'getting', 'harder', 'get', 'bed'...",0


In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile("[^0-9a-z #+_]")
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
dfnew['text'] = dfnew['text'].apply(clean_text)
df_holdout['text'] = df_holdout['text'].apply(clean_text)

dfnew.to_csv('master_f.csv', index=False)
df_holdout.to_csv('master_f_holdout.csv', index=False)

print(dfnew.head())
print(df_holdout.head())


X = dfnew['text'].astype(str).dropna()
y = dfnew['label']

vectorizer = TfidfVectorizer(
    stop_words='english', 
    min_df = 5,
    ngram_range = (1,2),
    use_idf = True,
    max_df = 0.5, 
    smooth_idf=True)

X = vectorizer.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2, 
    shuffle=True,
    random_state=42
)

                                                  text  label
100  struggling sad really dark thought almost 2yea...      0
101  something broke inside never gonna alright dre...      0
102  get anything done motivation wish world would ...      0
103  go college make parent happy want acknowledge ...      0
104  maybe genetics maybe childhood trauma maybe lo...      0
                                                text  label
0  know 16 people eye seems like easy get sorta p...      0
1  every moment sit pain sadness another moment l...      0
2  ever downplay anyone struggle especially come ...      0
3  struggling bad get bed sleep spend time watchi...      0
4  even know looking posting comment whatever lik...      0


In [23]:
model_dict={
    'MNB': MultinomialNB(),
    'SGD': SGDClassifier(),
    'LOG': LogisticRegression(),
    'RF': RandomForestClassifier(),
    'ADA': AdaBoostClassifier(),
    'GNB': GaussianNB(),
    'DT': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'DUM': DummyClassifier(),
}

def evaluate(model_dict, X_train, X_test, y_train, y_test):
    
    for k, v in model_dict.items():
        model = v
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

       
        print(k)
        print(classification_report(y_test, y_pred))
        
evaluate(model_dict, X_train, X_test, y_train, y_test)

MNB
              precision    recall  f1-score   support

           0       0.78      0.76      0.77       581
           1       0.72      0.74      0.73       593
           2       0.68      0.72      0.70       611
           3       0.45      0.84      0.58       597
           4       0.97      0.17      0.29       543
           5       0.86      0.71      0.78       600

    accuracy                           0.66      3525
   macro avg       0.74      0.65      0.64      3525
weighted avg       0.74      0.66      0.65      3525

SGD
              precision    recall  f1-score   support

           0       0.78      0.80      0.79       581
           1       0.84      0.77      0.80       593
           2       0.79      0.74      0.76       611
           3       0.88      0.79      0.83       597
           4       0.67      0.92      0.78       543
           5       0.87      0.78      0.82       600

    accuracy                           0.80      3525
   macro avg   

C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LOG
              precision    recall  f1-score   support

           0       0.74      0.77      0.76       581
           1       0.81      0.73      0.77       593
           2       0.74      0.70      0.72       611
           3       0.83      0.76      0.79       597
           4       0.69      0.89      0.78       543
           5       0.81      0.76      0.78       600

    accuracy                           0.77      3525
   macro avg       0.77      0.77      0.77      3525
weighted avg       0.77      0.77      0.77      3525

RF
              precision    recall  f1-score   support

           0       0.69      0.85      0.76       581
           1       0.88      0.79      0.83       593
           2       0.84      0.77      0.80       611
           3       0.85      0.83      0.84       597
           4       0.77      0.90      0.83       543
           5       0.93      0.78      0.85       600

    accuracy                           0.82      3525
   macro avg    

C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
